In [2]:
import sys
import os
from pathlib import Path

# Add src directory to path
project_root = os.path.join(str(Path.cwd().parent ),'src')
sys.path.insert(0, project_root)

# config
from utils import Config
config = Config(environment = "local")
print(config)

2025-08-02 09:21:46,057 - root - INFO - ✅ Loaded environment file for 'local' environment
2025-08-02 09:21:46,058 - root - INFO - 🔧 Logging configured at level: DEBUG
2025-08-02 09:21:46,058 - root - INFO - ✅ All required configuration loaded for 'local' environment


Config(log_level='DEBUG', environment='local', openai_api_key=None, openai_model=None, openai_api_type='azure', azure_openai_endpoint='https://aoai-zvbgv7oohofri.cognitiveservices.azure.com/', azure_openai_api_key=***, azure_openai_model='gpt-4o', azure_openai_deployment='gpt-4o', azure_openai_api_version='2024-12-01-preview', azure_openai_embedding_endpoint='https://aoai-zvbgv7oohofri.cognitiveservices.azure.com/', azure_openai_embedding_api_key=***, azure_openai_embedding_model='text-embedding-3-small', azure_openai_embedding_deployment='text-embedding-3-small', azure_openai_embedding_api_version='2024-12-01-preview', chroma_db_path='/data/chroma_db', serp_api_key=***, project_root=WindowsPath('c:/Users/aprilhazel/Source/sk_mcp_demo/mcp_rag'))


Creating the ChromaDB vector database, embedding, and storing data

In [3]:
import chromadb
from chromadb.utils import embedding_functions

# ChromaDB embeding_function
aoai_embedding_function = embedding_functions.OpenAIEmbeddingFunction(
    api_key=config.azure_openai_embedding_api_key,
    api_base=config.azure_openai_embedding_endpoint,
    api_type=config.openai_api_type,
    api_version=config.azure_openai_embedding_api_version,
    model_name=config.azure_openai_embedding_model,
    deployment_id=config.azure_openai_embedding_deployment
)

In [4]:
from pathlib import Path

chroma_db_path = Path(config.project_root/config.chroma_db_path.lstrip('./')/'chroma_db')
chroma_client = chromadb.PersistentClient(path=str(chroma_db_path))
print(f"ChromaDB path: {chroma_db_path}")

2025-08-02 09:22:15,522 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-08-02 09:22:16,526 - chromadb.config - DEBUG - Starting component System
2025-08-02 09:22:16,526 - chromadb.config - DEBUG - Starting component Posthog


ChromaDB path: c:\Users\aprilhazel\Source\sk_mcp_demo\mcp_rag\data\chroma_db\chroma_db


Create and a collection within the db

In [5]:
# Create or get a second collection
try:
    names_collection = chroma_client.get_or_create_collection (
        name="names_collection", embedding_function=aoai_embedding_function)
except Exception as e:
    raise e

chroma_client.list_collections()

[Collection(name=names_collection)]

2025-08-02 09:22:23,411 - urllib3.connectionpool - DEBUG - https://us.i.posthog.com:443 "POST /batch/ HTTP/1.1" 200 15


In [6]:
# Create a list of names with categories and popularity
names_data = [
    # People - High Popularity
    {"id": 1, "name": "Alexander", "category": "people", "popularity": 9},
    {"id": 2, "name": "Emma", "category": "people", "popularity": 10},
    {"id": 3, "name": "William", "category": "people", "popularity": 8},
    {"id": 4, "name": "Olivia", "category": "people", "popularity": 9},
    {"id": 5, "name": "James", "category": "people", "popularity": 8},
    {"id": 6, "name": "Sophia", "category": "people", "popularity": 9},
    {"id": 7, "name": "Michael", "category": "people", "popularity": 7},
    {"id": 8, "name": "Isabella", "category": "people", "popularity": 8},
    {"id": 9, "name": "Benjamin", "category": "people", "popularity": 7},
    {"id": 10, "name": "Charlotte", "category": "people", "popularity": 8},
    
    # People - Medium Popularity
    {"id": 11, "name": "Theodore", "category": "people", "popularity": 6},
    {"id": 12, "name": "Amelia", "category": "people", "popularity": 6},
    {"id": 13, "name": "Sebastian", "category": "people", "popularity": 5},
    {"id": 14, "name": "Penelope", "category": "people", "popularity": 5},
    {"id": 15, "name": "Nathaniel", "category": "people", "popularity": 4},
    {"id": 16, "name": "Genevieve", "category": "people", "popularity": 4},
    {"id": 17, "name": "Maximilian", "category": "people", "popularity": 3},
    {"id": 18, "name": "Evangeline", "category": "people", "popularity": 3},
    
    # Places - High Popularity
    {"id": 19, "name": "New York", "category": "places", "popularity": 10},
    {"id": 20, "name": "London", "category": "places", "popularity": 9},
    {"id": 21, "name": "Paris", "category": "places", "popularity": 9},
    {"id": 22, "name": "Tokyo", "category": "places", "popularity": 8},
    {"id": 23, "name": "Sydney", "category": "places", "popularity": 7},
    {"id": 24, "name": "Rome", "category": "places", "popularity": 8},
    {"id": 25, "name": "Barcelona", "category": "places", "popularity": 7},
    {"id": 26, "name": "Amsterdam", "category": "places", "popularity": 6},
    
    # Places - Medium Popularity
    {"id": 27, "name": "Reykjavik", "category": "places", "popularity": 4},
    {"id": 28, "name": "Budapest", "category": "places", "popularity": 5},
    {"id": 29, "name": "Prague", "category": "places", "popularity": 6},
    {"id": 30, "name": "Vienna", "category": "places", "popularity": 5},
    {"id": 31, "name": "Stockholm", "category": "places", "popularity": 4},
    {"id": 32, "name": "Copenhagen", "category": "places", "popularity": 4},
    {"id": 33, "name": "Helsinki", "category": "places", "popularity": 3},
    {"id": 34, "name": "Tallinn", "category": "places", "popularity": 2},
    
    # Things - High Popularity
    {"id": 35, "name": "Smartphone", "category": "things", "popularity": 10},
    {"id": 36, "name": "Laptop", "category": "things", "popularity": 9},
    {"id": 37, "name": "Coffee", "category": "things", "popularity": 10},
    {"id": 38, "name": "Bicycle", "category": "things", "popularity": 7},
    {"id": 39, "name": "Camera", "category": "things", "popularity": 6},
    {"id": 40, "name": "Headphones", "category": "things", "popularity": 8},
    {"id": 41, "name": "Backpack", "category": "things", "popularity": 7},
    {"id": 42, "name": "Sunglasses", "category": "things", "popularity": 6},
    
    # Things - Medium Popularity
    {"id": 43, "name": "Telescope", "category": "things", "popularity": 3},
    {"id": 44, "name": "Microscope", "category": "things", "popularity": 2},
    {"id": 45, "name": "Synthesizer", "category": "things", "popularity": 4},
    {"id": 46, "name": "Skateboard", "category": "things", "popularity": 5},
    {"id": 47, "name": "Harmonica", "category": "things", "popularity": 3},
    {"id": 48, "name": "Kaleidoscope", "category": "things", "popularity": 2},
    {"id": 49, "name": "Binoculars", "category": "things", "popularity": 4},
    {"id": 50, "name": "Compass", "category": "things", "popularity": 3},
    
    # Fictional Characters - High Popularity
    {"id": 51, "name": "Sherlock Holmes", "category": "fictional", "popularity": 9},
    {"id": 52, "name": "Harry Potter", "category": "fictional", "popularity": 10},
    {"id": 53, "name": "Batman", "category": "fictional", "popularity": 9},
    {"id": 54, "name": "Wonder Woman", "category": "fictional", "popularity": 8},
    {"id": 55, "name": "Spider-Man", "category": "fictional", "popularity": 9},
    {"id": 56, "name": "Hermione Granger", "category": "fictional", "popularity": 8},
    {"id": 57, "name": "Luke Skywalker", "category": "fictional", "popularity": 8},
    {"id": 58, "name": "Princess Leia", "category": "fictional", "popularity": 7},
    
    # Fictional Characters - Medium Popularity
    {"id": 59, "name": "Gandalf", "category": "fictional", "popularity": 6},
    {"id": 60, "name": "Aragorn", "category": "fictional", "popularity": 5},
    {"id": 61, "name": "Legolas", "category": "fictional", "popularity": 4},
    {"id": 62, "name": "Gimli", "category": "fictional", "popularity": 3},
    {"id": 63, "name": "Frodo Baggins", "category": "fictional", "popularity": 6},
    {"id": 64, "name": "Samwise Gamgee", "category": "fictional", "popularity": 5},
    
    # Animals - High Popularity
    {"id": 65, "name": "Golden Retriever", "category": "animals", "popularity": 9},
    {"id": 66, "name": "Siamese Cat", "category": "animals", "popularity": 7},
    {"id": 67, "name": "German Shepherd", "category": "animals", "popularity": 8},
    {"id": 68, "name": "Persian Cat", "category": "animals", "popularity": 6},
    {"id": 69, "name": "Labrador", "category": "animals", "popularity": 9},
    {"id": 70, "name": "Maine Coon", "category": "animals", "popularity": 5},
    
    # Animals - Medium Popularity
    {"id": 71, "name": "Border Collie", "category": "animals", "popularity": 6},
    {"id": 72, "name": "Scottish Fold", "category": "animals", "popularity": 4},
    {"id": 73, "name": "Siberian Husky", "category": "animals", "popularity": 7},
    {"id": 74, "name": "Bengal Cat", "category": "animals", "popularity": 4},
    {"id": 75, "name": "Australian Shepherd", "category": "animals", "popularity": 5},
    {"id": 76, "name": "Ragdoll Cat", "category": "animals", "popularity": 3},
]


In [7]:
# Prepare data for ChromaDB insertion
names_contents = []
names_metadatas = []
names_ids = []

for name_data in names_data:
    # Create a content string that includes all information about the name
    content_text = f"ID: {name_data['id']}, Name: {name_data['name']}, Category: {name_data['category']}, Popularity: {name_data['popularity']}/10"
    
    # Create metadata with id, category and popularity
    metadata = {
        'id': name_data['id'],
        'name': name_data['name'],
        'category': name_data['category'],
        'popularity': name_data['popularity']
    }
    
    names_contents.append(content_text)
    names_metadatas.append(metadata)
    names_ids.append(str(name_data['id']))  # Use the id field as ChromaDB content ID

# Insert data into ChromaDB names collection
names_collection.upsert(
    documents=names_contents,
    metadatas=names_metadatas,
    ids=names_ids
)

print(f"Successfully upserted {len(names_contents)} names into the names ChromaDB collection.")
print(f"Names collection now contains {names_collection.count()} contents.")

2025-08-02 09:22:29,505 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/embeddings', 'headers': {'api-key': '<redacted>'}, 'files': None, 'idempotency_key': 'stainless-python-retry-8b68e9fd-2430-486f-84dd-169cdc9da0b0', 'post_parser': <function Embeddings.create.<locals>.parser at 0x00000154DFB034C0>, 'json_data': {'input': ['ID: 1, Name: Alexander, Category: people, Popularity: 9/10', 'ID: 2, Name: Emma, Category: people, Popularity: 10/10', 'ID: 3, Name: William, Category: people, Popularity: 8/10', 'ID: 4, Name: Olivia, Category: people, Popularity: 9/10', 'ID: 5, Name: James, Category: people, Popularity: 8/10', 'ID: 6, Name: Sophia, Category: people, Popularity: 9/10', 'ID: 7, Name: Michael, Category: people, Popularity: 7/10', 'ID: 8, Name: Isabella, Category: people, Popularity: 8/10', 'ID: 9, Name: Benjamin, Category: people, Popularity: 7/10', 'ID: 10, Name: Charlotte, Category: people, Popularity: 8/10', 'ID: 11, Name: Theodore, Category: people, 

Successfully upserted 76 names into the names ChromaDB collection.
Names collection now contains 76 contents.


Let's do a chroma search

In [8]:
def search_chroma(collection, query, n_results=5):
    """Search the ChromaDB collection for the given query."""
    try:
        # Get ChromaDB information
        chroma_info = {
            'db_path': str(chroma_db_path),
            'collection_name': collection.name,
            'collection_count': collection.count()
        }

        # Search the collection with the query
        search_results = collection.query(
            query_texts=[query],
            n_results=n_results,
            include=['documents', 'metadatas', 'distances']
        )
        
        # Format the retrieved context
        context_contents = search_results['documents'][0] if search_results['documents'] else []
        context_metadata = search_results['metadatas'][0] if search_results['metadatas'] else []
        # Get IDs separately - ChromaDB always returns IDs
        context_ids = search_results.get('ids', [[]])[0] if 'ids' in search_results else []
        
        # If no IDs in search results, we need to get them differently
        if not context_ids and context_contents:
            # Use index-based IDs as fallback
            context_ids = [f"doc_{i}" for i in range(len(context_contents))]

        structured_context = []
        
        if context_contents:  # Only build if we have contents
            for i, (doc, metadata) in enumerate(zip(context_contents, context_metadata)):
                # Get content ID, using index as fallback
                doc_id = context_ids[i] if i < len(context_ids) else f"doc_{i}"
                
                # Create formatted citation
                citation = f"[Source: ChromaDB | Path: {chroma_info['db_path']} | Collection: {chroma_info['collection_name']} | content ID: {doc_id}]"
                
                # Create structured result object
                result_obj = {
                    'search_order': i + 1,
                    'content': doc,
                    'citation': citation,
                    'metadata': metadata,
                    'context_id': doc_id
                }
                structured_context.append(result_obj)

        return structured_context, chroma_info
        
    except Exception as e:
        # Always return a tuple to avoid unpacking errors
        return [], {'error': str(e)}
    
# Test the search function
search_query = "What are the most popular names?"
search_results, chroma_info = search_chroma(names_collection, search_query, n_results=5)
print("###############################")
print("🔍 Search Results:")
print(f"Query: {search_query}")
print(f"ChromaDB Info: {chroma_info}")

if len(search_results) >= 0:
    print(f"Found {len(search_results)} results\n")
    for result in search_results:
        print(f"--- Result #{result['search_order']} ---")
        print(f"content: {result['content'][:100]}...")
        print(f"Citation: {result['citation']}")
else:
    print(f"Error or no results: {search_results}")

2025-08-02 09:22:30,839 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/embeddings', 'headers': {'api-key': '<redacted>'}, 'files': None, 'idempotency_key': 'stainless-python-retry-b4b5edf0-1318-4e88-847c-9fd0241fabc4', 'post_parser': <function Embeddings.create.<locals>.parser at 0x00000154DF653880>, 'json_data': {'input': ['What are the most popular names?'], 'model': 'text-embedding-3-small', 'encoding_format': 'base64'}}
2025-08-02 09:22:30,841 - openai._base_client - DEBUG - Sending HTTP Request: POST https://aoai-zvbgv7oohofri.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview
2025-08-02 09:22:30,842 - httpcore.http11 - DEBUG - send_request_headers.started request=<Request [b'POST']>
2025-08-02 09:22:30,843 - httpcore.http11 - DEBUG - send_request_headers.complete
2025-08-02 09:22:30,844 - httpcore.http11 - DEBUG - send_request_body.started request=<Request [b'POST']>
2025-08-02 09:22:30,844

###############################
🔍 Search Results:
Query: What are the most popular names?
ChromaDB Info: {'db_path': 'c:\\Users\\aprilhazel\\Source\\sk_mcp_demo\\mcp_rag\\data\\chroma_db\\chroma_db', 'collection_name': 'names_collection', 'collection_count': 76}
Found 5 results

--- Result #1 ---
content: ID: 9, Name: Benjamin, Category: people, Popularity: 7/10...
Citation: [Source: ChromaDB | Path: c:\Users\aprilhazel\Source\sk_mcp_demo\mcp_rag\data\chroma_db\chroma_db | Collection: names_collection | content ID: 9]
--- Result #2 ---
content: ID: 1, Name: Alexander, Category: people, Popularity: 9/10...
Citation: [Source: ChromaDB | Path: c:\Users\aprilhazel\Source\sk_mcp_demo\mcp_rag\data\chroma_db\chroma_db | Collection: names_collection | content ID: 1]
--- Result #3 ---
content: ID: 4, Name: Olivia, Category: people, Popularity: 9/10...
Citation: [Source: ChromaDB | Path: c:\Users\aprilhazel\Source\sk_mcp_demo\mcp_rag\data\chroma_db\chroma_db | Collection: names_collection | cont

2025-08-02 09:22:31,125 - urllib3.connectionpool - DEBUG - https://us.i.posthog.com:443 "POST /batch/ HTTP/1.1" 200 15
